In [18]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4

cuda


In [19]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)

In [20]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l)

data = torch.tensor(encode(text), dtype=torch.long)

In [26]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs: ', x)
print('targets: ', y)

inputs:  tensor([[60, 62, 75, 82,  1, 77, 62, 69],
        [63, 75, 78, 66, 77,  1, 80, 58],
        [71, 64,  1, 76, 77, 58, 66, 75],
        [62, 76, 14,  1, 29, 76,  1, 76]], device='cuda:0')
targets:  tensor([[62, 75, 82,  1, 77, 62, 69, 69],
        [75, 78, 66, 77,  1, 80, 58, 76],
        [64,  1, 76, 77, 58, 66, 75, 80],
        [76, 14,  1, 29, 76,  1, 76, 65]], device='cuda:0')


In [27]:
# --- Rerun these lines after fixing vocab_size ---
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self.forward(index)
            logits = logits[:, -1, :]  # (B, C)
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

# Recreate model
model = BigramLanguageModel(vocab_size)
m = model.to(device)

In [28]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


hIVi‘—k kN#JbI&﻿JMwI:Yt,N4KP”O6N,Hm[(FB—?—Yn".iWp
N725•E
“&lP—Db(—6EO3u6S$™"0je&He)qPG;v—cvpQ8zHmr$”na:;Fk&]c:EU-0dp*,2WiRrY0﻿8cwknrW]z”;uC[/2PG
•RpLxU[a)%J“u%V&H™W'Q$n•Z%?9 m1S04h(NQGDIIwkxJcE%F,•T_nUu0.c*’8.I“.uhmDVQ$rAIv‘,/wkHa&m3p
[!0r6kzeq2kGLNy﻿Tr—T‘%vx:X”)or/T
“UK:J“[YY,﻿l™flF””d’k‘CR]U?A8W]K%:Ip9 YDA:Ia.5 S’8;Xa-(*FhcXWK:,HnH‘ OkyrQK”3eZs*(q9(8Rw"c﻿k69qL%•;GQn!J/4hqLCR)N4ZX5&-JT1]20T-'/$4g [,[T•KIC2VxyDZ-,C[HmBdwFC8u5M(*
;﻿u6;Vu“u%r6C‘i6z:IU” S™-—heN.HtO#1﻿.*dqI•0v‘.IGU(3/K[q5nU?faF sJK(


In [29]:
# pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.0370893478393555
